<a href="https://colab.research.google.com/github/franckoj/pyspark/blob/main/usecase_7_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install pyspark
 
 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as F
from pyspark.sql.types import *
spark=SparkSession.builder.appName('UseCase').getOrCreate()
 
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

In [ ]:
schema_item = 'movie_id long, movie_title string,releasedate string, videoreleasedate long,IMDbURL string, unknown string , Action string, Adventure string, Animation string,Childrens string, Comedy string, Crime string, Documentary string, Drama string, Fantasy string,FilmNoir string, Horror string, Musical string, Mystery string, Romance string, SciFi string,Thriller string, War string, Western string'
df_item = spark.read.csv('/content/ml-100k/u.item',sep='|',schema=schema_item)\
.withColumn('year',regexp_extract(col('movie_title'),'(\d{4})',1)).drop('videoreleasedate','IMDbURL','releasedate')
#.select('movie_id','movie_title','year')
 
schema_data = 'user_id string, item_id long, rating int, timestamp bigint'
df_data = spark.read.csv('/content/ml-100k/u.data',sep='\t', schema=schema_data)\
.withColumn('date',from_unixtime('timestamp', format='yyyy-MM-dd'))\
.select('item_id','rating','date')

In [ ]:
df_item.groupBy('year','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western')\
.agg(count('Action').alias('sum'),max('year')).show()

In [ ]:
list = ['year','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western']
l = df_item.groupBy(list)\
.count().alias('uu')

In [ ]:
l.orderBy(['year','count'], ascending = False).dropDuplicates(subset=['year']).sort(asc('year')).repartition(1).write.option('header' ,True).csv("mov")

In [ ]:
l.filter('Action ==1').count()

169

In [ ]:
l.join(df_item,list).groupBy('year').count().filter('year==1995').show()

+----+-----+
|year|count|
+----+-----+
|1995|  296|
+----+-----+



In [ ]:
l.groupBy().sum().show()

+-------+
|sum(uu)|
+-------+
|   1682|
+-------+



In [ ]:
l.orderBy('Action', ascending =False).show()

NameError: ignored

In [ ]:
rating = df_data.groupBy('item_id')\
.agg(avg('rating').alias('average_rating'),count('rating').alias('no_of_ratings'))

In [ ]:
 
#joining the the ratings table and movie table
d = df_item.join(rating,df_item.movie_id == rating.item_id)

In [ ]:
##groupby year and genres,and then order by average rating(descending) and drop the duplicates form the year 
d.groupBy('year','movie_id','average_rating','no_of_ratings','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','FilmNoir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western')\
.agg(count('movie_id'))\
#.orderBy(desc('average_rating')).drop_duplicates(subset=['year']).drop('count(movie_id)').sort(asc('year'))\
.repartition(1).write.option('header',True).csv('mo')